In [ ]:
import numpy as np
import torch    
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
#data_dir = 'DATASET'

# TODO: Define transforms for the training data and testing data
train_transforms =transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
#train_data = datasets.ImageFolder(data_dir + '/TRAIN', transform=train_transforms)
#test_data = datasets.ImageFolder(data_dir + '/TEST', transform=test_transforms)

classes = ['Organic', 'Inorganic']

#now check if they've loaded correctly
print("Number of train images: ", (len(train_data)))
print("Number of test images: ", len(test_data))

In [ ]:
batch_size = 1

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [ ]:
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.features.parameters():
    param.requires_grad = False

# access the last layer in the net
n_inputs = vgg16.classifier[6].in_features

# create new layer to have original number of in_features and out_features equal to number of classes we have in our data
last_layer = nn.Linear(n_inputs, len(classes))

# overwrite last layer with our layer
vgg16.classifier[6] = last_layer

# check if it's correct
print(vgg16)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001)

In [ ]:
n_epochs = 1

for epoch in range(n_epochs):
    #keep track of training loss
    train_loss = 0
    
    for batch_i, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()             #clear the gradients
        output = vgg16(data)              #do the forward pass
        loss = criterion(output, target)  #calculate loss
        loss.backward()                   #do the backward pass
        optimizer.step()                  #perform parameter update
        train_loss += loss.item()         #update training loss
        
        if batch_i:    # print training loss every specified number of mini-batches
            print('Epoch %d, Batch %d loss: %.16f' %
                  (epoch, batch_i + 1, train_loss / 20))
            train_loss = 0.0